In [1]:
from __future__ import print_function
import paddle
import paddle.fluid as fluid
import numpy as np
import sys
import math

CLASS_DIM = 2 #Number of categories for sentiment analysis
EMB_DIM = 64 #Dimensions of the word vector
HID_DIM = 64 #Dimensions of hide layer
STACKED_NUM = 2 #LSTM Layers of the bidirectional stack
BATCH_SIZE = 64 #batch size

In [2]:
#Textconvolution neural network
def convolution_net(data, input_dim, class_dim, emb_dim, hid_dim):
    emb = fluid.layers.embedding(
        input=data, size=[input_dim, emb_dim], is_sparse=True)
    conv_3 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,
        filter_size=3,
        act="tanh",
        pool_type="sqrt")
    conv_4 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,
        filter_size=4,
        act="tanh",
        pool_type="sqrt")
    prediction = fluid.layers.fc(
        input=[conv_3, conv_4], size=class_dim, act="softmax")
    return prediction

In [3]:
#Stack Bidirectional LSTM
def stacked_lstm_net(data, input_dim, class_dim, emb_dim, hid_dim, stacked_num):

    # Calculate word vectorvector
    emb = fluid.layers.embedding(input=data, size=[input_dim, emb_dim], is_sparse=True)

    #First stack
    #Fully connected layer
    fc1 = fluid.layers.fc(input=emb, size=hid_dim)
    #lstm layer
    lstm1, cell1 = fluid.layers.dynamic_lstm(input=fc1, size=hid_dim)

    inputs = [fc1, lstm1]

    #All remaining stack structures
    for i in range(2, stacked_num + 1):
        fc = fluid.layers.fc(input=inputs, size=hid_dim)
        lstm, cell = fluid.layers.dynamic_lstm(input=fc, size=hid_dim, is_reverse=(i % 2) == 0)
        inputs = [fc, lstm]

    #pooling layer
    pc_last = fluid.layers.sequence_pool(input=inputs[0], pool_type='max')
    lstm_last = fluid.layers.sequence_pool(input=inputs[1], pool_type='max')

    #Fully connected layer, softmax prediction
    prediction = fluid.layers.fc(
        input=[fc_last, lstm_last], size=class_dim, act='softmax')
    return prediction

In [4]:
def inference_program(word_dict):
    data = fluid.layers.data(
        name="words", shape=[1], dtype="int64", lod_level=1)

    dict_dim = len(word_dict)
    net = convolution_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM)
    # net = stacked_lstm_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM, STACKED_NUM)
    return net

In [5]:
def train_program(prediction):
    label = fluid.layers.data(name="label", shape=[1], dtype="int64")
    cost = fluid.layers.cross_entropy(input=prediction, label=label)
    avg_cost = fluid.layers.mean(cost)
    accuracy = fluid.layers.accuracy(input=prediction, label=label)
    return [avg_cost, accuracy] #return average cost and accuracy acc

#Optimization function
def optimizer_func():
    return fluid.optimizer.Adagrad(learning_rate=0.002)

In [6]:
use_cuda = True 
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

In [7]:
print("Loading IMDB word dict....")
word_dict = paddle.dataset.imdb.word_dict()

print ("Reading training data....")
train_reader = paddle.batch(
    paddle.reader.shuffle(
        paddle.dataset.imdb.train(word_dict), buf_size=25000),
    batch_size=BATCH_SIZE)
print("Reading testing data....")
test_reader = paddle.batch(
    paddle.dataset.imdb.test(word_dict), batch_size=BATCH_SIZE)

Loading IMDB word dict....


[==================================================]3563afcf9b574c7e56c153aCache file /root/.cache/paddle/dataset/imdb/aclImdb_v1.tar.gz not found, downloading http://ai.stanford.edu/%7Eamaas/data/sentiment/aclImdb_v1.tar.gz


Reading training data....


Reading testing data....


In [8]:
word_dict

{b'the': 0,
 b'and': 1,
 b'a': 2,
 b'of': 3,
 b'to': 4,
 b'is': 5,
 b'in': 6,
 b'it': 7,
 b'i': 8,
 b'this': 9,
 b'that': 10,
 b'br': 11,
 b'was': 12,
 b'as': 13,
 b'with': 14,
 b'for': 15,
 b'movie': 16,
 b'but': 17,
 b'film': 18,
 b'on': 19,
 b'not': 20,
 b'you': 21,
 b'are': 22,
 b'his': 23,
 b'have': 24,
 b'be': 25,
 b'he': 26,
 b'one': 27,
 b'its': 28,
 b'at': 29,
 b'all': 30,
 b'by': 31,
 b'an': 32,
 b'they': 33,
 b'from': 34,
 b'who': 35,
 b'so': 36,
 b'like': 37,
 b'or': 38,
 b'just': 39,
 b'her': 40,
 b'about': 41,
 b'if': 42,
 b'has': 43,
 b'out': 44,
 b'some': 45,
 b'there': 46,
 b'what': 47,
 b'good': 48,
 b'when': 49,
 b'more': 50,
 b'very': 51,
 b'my': 52,
 b'even': 53,
 b'no': 54,
 b'up': 55,
 b'would': 56,
 b'she': 57,
 b'time': 58,
 b'only': 59,
 b'which': 60,
 b'really': 61,
 b'their': 62,
 b'see': 63,
 b'were': 64,
 b'story': 65,
 b'had': 66,
 b'can': 67,
 b'me': 68,
 b'than': 69,
 b'we': 70,
 b'much': 71,
 b'well': 72,
 b'been': 73,
 b'get': 74,
 b'will': 75,
 b'oth

In [9]:
exe = fluid.Executor(place)
prediction = inference_program(word_dict)
[avg_cost, accuracy] = train_program(prediction)#training program
sgd_optimizer = optimizer_func()# training optimization function
sgd_optimizer.minimize(avg_cost)

([inputs {
    parameter: "Grad"
    arguments: "embedding_0.w_0@GRAD"
  }
  inputs {
    parameter: "LearningRate"
    arguments: "learning_rate_0"
  }
  inputs {
    parameter: "Moment"
    arguments: "embedding_0.w_0_moment_0"
  }
  inputs {
    parameter: "Param"
    arguments: "embedding_0.w_0"
  }
  outputs {
    parameter: "MomentOut"
    arguments: "embedding_0.w_0_moment_0"
  }
  outputs {
    parameter: "ParamOut"
    arguments: "embedding_0.w_0"
  }
  type: "adagrad"
  attrs {
    name: "op_callstack"
    type: STRINGS
    strings: "  File \"/usr/local/lib/python3.6/dist-packages/paddle/fluid/framework.py\", line 1748, in append_op\n    attrs=kwargs.get(\"attrs\", None))\n"
    strings: "  File \"/usr/local/lib/python3.6/dist-packages/paddle/fluid/optimizer.py\", line 1228, in _append_optimize_op\n    stop_gradient=True)\n"
    strings: "  File \"/usr/local/lib/python3.6/dist-packages/paddle/fluid/optimizer.py\", line 386, in _create_optimization_pass\n    param_and_grad)\n"

In [10]:
def train_test(program, reader):
    count = 0
    feed_var_list = [
        program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder_test = fluid.DataFeeder(feed_list=feed_var_list, place=place)
    test_exe = fluid.Executor(place)
    accumulated = len([avg_cost, accuracy]) * [0]
    for test_data in reader():
        avg_cost_np = test_exe.run(
            program=program,
            feed=feeder_test.feed(test_data),
            fetch_list=[avg_cost, accuracy])
        accumulated = [
            x[0] + x[1][0] for x in zip(accumulated, avg_cost_np)
        ]
        count += 1
    return [x / count for x in accumulated]

In [12]:
# Specify the directory path to save the parameters
params_dirname = "../model/understand_sentiment_conv.inference.model"

feed_order = ['words', 'label']
pass_num = 20  #Number rounds of the training loop

# Main loop part of the program
def train_loop(main_program):
    # Start the trainer built above
    exe.run(fluid.default_startup_program())

    feed_var_list_loop = [
        main_program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder = fluid.DataFeeder(
        feed_list=feed_var_list_loop, place=place)

    test_program = fluid.default_main_program().clone(for_test=True)

    # Training loop
    for epoch_id in range(pass_num):
        for step_id, data in enumerate(train_reader()):
            # Running trainer
            metrics = exe.run(main_program,
                              feed=feeder.feed(data),
                              fetch_list=[avg_cost, accuracy])

            # Testing Results
            avg_cost_test, acc_test = train_test(test_program, test_reader)
            print('Step {0}, Test Loss {1:0.2}, Acc {2:0.2}'.format(
                step_id, avg_cost_test, acc_test))

            print("Step {0}, Epoch {1} Metrics {2}".format(
                step_id, epoch_id, list(map(np.array,
                                            metrics))))

            if step_id == 30:
                if params_dirname is not None:
                    fluid.io.save_inference_model(params_dirname, ["words"],
                                                  prediction, exe)# Save model
                return

In [13]:
train_loop(fluid.default_main_program())


EnforceNotMet: Invoke operator fill_constant error.
Python Callstacks: 
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/framework.py", line 1819, in _prepend_op
    attrs=kwargs.get("attrs", None))
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/initializer.py", line 186, in __call__
    stop_gradient=True)
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/framework.py", line 1602, in create_var
    kwargs['initializer'](var, self)
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/layer_helper_base.py", line 383, in set_variable_initializer
    initializer=initializer)
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/layers/tensor.py", line 139, in create_global_var
    value=float(value), force_cpu=force_cpu))
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/optimizer.py", line 226, in _create_global_learning_rate
    persistable=True)
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/optimizer.py", line 365, 